In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, classification_report, confusion_matrix

data = pd.read_csv("boston.csv")  
data.dropna(inplace=True) 

scaler = StandardScaler()
X = scaler.fit_transform(data.drop(columns=["MEDV"]))  
y_reg = data["MEDV"].values  

median_price = np.median(y_reg)
y_cls = np.where(y_reg >= median_price, 1, 0)  

X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X, y_reg, test_size=0.2, random_state=42)
X_train_cls, X_test_cls, y_train_cls, y_test_cls = train_test_split(X, y_cls, test_size=0.2, random_state=42)

knn_reg = KNeighborsRegressor(n_neighbors=5)
knn_reg.fit(X_train_reg, y_train_reg)
y_pred_reg = knn_reg.predict(X_test_reg)

knn_cls = KNeighborsClassifier(n_neighbors=5)
knn_cls.fit(X_train_cls, y_train_cls)
y_pred_cls = knn_cls.predict(X_test_cls)

mse_initial = mean_squared_error(y_test_reg, y_pred_reg)
r2_initial = r2_score(y_test_reg, y_pred_reg)
accuracy_initial = accuracy_score(y_test_cls, y_pred_cls)

kf = KFold(n_splits=5, shuffle=True, random_state=42)
param_grid = {"n_neighbors": range(1, 21)}

grid_reg = GridSearchCV(KNeighborsRegressor(), param_grid, cv=kf)
grid_reg.fit(X_train_reg, y_train_reg)
best_k_reg = grid_reg.best_params_["n_neighbors"]

grid_cls = GridSearchCV(KNeighborsClassifier(), param_grid, cv=kf)
grid_cls.fit(X_train_cls, y_train_cls)
best_k_cls = grid_cls.best_params_["n_neighbors"]

knn_reg_opt = KNeighborsRegressor(n_neighbors=best_k_reg)
knn_reg_opt.fit(X_train_reg, y_train_reg)
y_pred_reg_opt = knn_reg_opt.predict(X_test_reg)

knn_cls_opt = KNeighborsClassifier(n_neighbors=best_k_cls)
knn_cls_opt.fit(X_train_cls, y_train_cls)
y_pred_cls_opt = knn_cls_opt.predict(X_test_cls)

mse_opt = mean_squared_error(y_test_reg, y_pred_reg_opt)
r2_opt = r2_score(y_test_reg, y_pred_reg_opt)
accuracy_opt = accuracy_score(y_test_cls, y_pred_cls_opt)

mse_diff = mse_initial - mse_opt
r2_diff = r2_opt - r2_initial
accuracy_diff = accuracy_opt - accuracy_initial

print(f"\n🔹 Різниця після оптимізації:")
print(f"MSE змінилось на: {mse_diff:.4f} (чим менше, тим краще)")
print(f"R2 Score змінилось на: {r2_diff:.4f} (чим більше, тим краще)")
print(f"Accuracy змінилось на: {accuracy_diff:.4f} (чим більше, тим краще)")

params_comparison = pd.DataFrame({
    "Модель": ["Початкова (Регресія)", "Оптимізована (Регресія)", "Початкова (Класифікація)", "Оптимізована (Класифікація)"],
    "n_neighbors": [5, best_k_reg, 5, best_k_cls],
    "MSE": [mse_initial, mse_opt, np.nan, np.nan],
    "R2 Score": [r2_initial, r2_opt, np.nan, np.nan],
    "Accuracy": [np.nan, np.nan, accuracy_initial, accuracy_opt]
})

print("\n🔹 Порівняння параметрів моделей:")
print(params_comparison)

plt.figure(figsize=(10, 5))
sns.scatterplot(x=y_test_reg, y=y_pred_reg, color='blue', label='Початкові передбачення')
sns.scatterplot(x=y_test_reg, y=y_pred_reg_opt, color='red', label='Оптимізовані передбачення')
plt.xlabel('Реальні значення')
plt.ylabel('Прогнозовані значення')
plt.title('k-NN Регресія: Реальні vs Прогнозовані (До і Після Оптимізації)')
plt.legend()
plt.show()

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

sns.heatmap(confusion_matrix(y_test_cls, y_pred_cls), annot=True, cmap='Blues', fmt='d', ax=axes[0])
axes[0].set_title('Матриця помилок (Початкова модель)')
axes[0].set_xlabel('Прогноз')
axes[0].set_ylabel('Реальне значення')

sns.heatmap(confusion_matrix(y_test_cls, y_pred_cls_opt), annot=True, cmap='Blues', fmt='d', ax=axes[1])
axes[1].set_title('Матриця помилок (Оптимізована модель)')
axes[1].set_xlabel('Прогноз')
axes[1].set_ylabel('Реальне значення')

plt.show()

print("\n🔹 Classification Report для початкової моделі класифікації:")
print(classification_report(y_test_cls, y_pred_cls))

print("\n🔹 Classification Report для оптимізованої моделі класифікації:")
print(classification_report(y_test_cls, y_pred_cls_opt))
